<a href="https://colab.research.google.com/github/quillaur/data_learning/blob/main/data_science/computer_vision/object_detction_using_image_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

## Dataset creation

# Using image AI

https://github.com/OlafenwaMoses/ImageAI/tree/master/imageai/Detection

In [6]:
! pip install imageai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 KB 7.8 MB/s eta 0:00:00


In [7]:
from imageai.Detection import ObjectDetection

In [8]:
detector = ObjectDetection()
detector.setModelTypeAsTinyYOLOv3()
detector.setModelPath("/content/tiny-yolov3.pt")
detector.loadModel()

ValueError: ignored

In [ ]:
input_img = "/content/other_4.jpeg"
out_img = "/content/other_4_out.jpeg"
detections = detector.detectObjectsFromImage(input_image=input_img, output_image_path=out_img, minimum_percentage_probability=30)

In [ ]:
for eachObject in detections:
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")